In [1]:
#install all the required libraries
!pip install -U -q pdfplumber tiktoken openai chromaDB sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 767.7/767.7 kB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.2/470.2 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 7.9 MB/s eta

In [17]:
#importing the required liberies
import pdfplumber
from pathlib import Path
import pandas as pd
from operator import itemgetter
import json
import tiktoken
import openai
import chromadb

In [18]:
pdf_path="/content/Principal-Sample-Life-Insurance-Policy.pdf"

In [19]:
# Function to extract text from a PDF file.
# 1. Declare a variable p to store the iteration of the loop that will help us store page numbers alongside the text
# 2. Declare an empty list 'full_text' to store all the text files
# 3. Use pdfplumber to open the pdf pages one by one
# 4. Find the tables and their locations in the page
# 5. Extract the text from the tables in the variable 'tables'
# 6. Extract the regular words by calling the function check_bboxes() and checking whether words are present in the table or not
# 7. Use the cluster_objects utility to cluster non-table and table words together so that they retain the same chronology as in the original PDF
# 8. Declare an empty list 'lines' to store the page text
# 9. If a text element in present in the cluster, append it to 'lines', else if a table element is present, append the table
# 10. Append the page number and all lines to full_text, and increment 'p'
# 11. When the function has iterated over all pages, return the 'full_text' list
def extract_text_from_pdf(pdf_path):
    p = 0
    full_text = []


    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page_no = f"Page {p+1}"
            text = page.extract_text()

            tables = page.find_tables()
            table_bboxes = [i.bbox for i in tables]
            tables = [{'table': i.extract(), 'top': i.bbox[1]} for i in tables]
            non_table_words = [word for word in page.extract_words() if not any(
                [check_bboxes(word, table_bbox) for table_bbox in table_bboxes])]
            lines = []

            for cluster in pdfplumber.utils.cluster_objects(non_table_words + tables, itemgetter('top'), tolerance=5):

                if 'text' in cluster[0]:
                    try:
                        lines.append(' '.join([i['text'] for i in cluster]))
                    except KeyError:
                        pass

                elif 'table' in cluster[0]:
                    lines.append(json.dumps(cluster[0]['table']))


            full_text.append([page_no, " ".join(lines)])
            p +=1

    return full_text

In [20]:
extract_text_from_pdf(pdf_path)

[['Page 1',
  'DOROTHEA GLAUSE S655 RHODE ISLAND JOHN DOE 01/01/2014 711 HIGH STREET GEORGE RI 02903 GROUP POLICY FOR: RHODE ISLAND JOHN DOE ALL MEMBERS Group Member Life Insurance Print Date: 07/16/2014'],
 ['Page 2', 'This page left blank intentionally'],
 ['Page 3',
  'POLICY RIDER GROUP INSURANCE POLICY NO: S655 COVERAGE: Life EMPLOYER: RHODE ISLAND JOHN DOE Effective on the later of the Date of Issue of this Group Policy or March 1, 2005, the following will apply to your Policy: From time to time The Principal may offer or provide certain employer groups who apply for coverage with The Principal a Financial Services Hotline and Grief Support Services or any other value added service for the employees of that employer group. In addition, The Principal may arrange for third party service providers (i.e., optometrists, health clubs), to provide discounted goods and services to those employer groups who apply for coverage with The Principal or who become insureds/enrollees of The Prin

In [24]:
# Define the path to the PDF file
pdf_file_path = Path(pdf_path)

# Initialize an empty list to store the extracted texts and document names
data = []

# Process the PDF file
print(f"...Processing {pdf_file_path.name}")

# Call the function to extract the text from the PDF
extracted_text = extract_text_from_pdf(pdf_file_path)

# Convert the extracted list to a DataFrame, and add a column to store document names
extracted_text_df = pd.DataFrame(extracted_text, columns=['Page No.', 'Page_Text'])
extracted_text_df['Document Name'] = pdf_file_path.name

# Append the extracted text and document name to the list
data.append(extracted_text_df)

# Print a message to indicate progress
print(f"Finished processing {pdf_file_path.name}")

# Print a message to indicate all PDFs have been processed
print("All PDFs have been processed.")

# Concatenate the dataframes in the list into a single dataframe
insurance_pdfs_data = pd.concat(data, ignore_index=True)

...Processing Principal-Sample-Life-Insurance-Policy.pdf
Finished processing Principal-Sample-Life-Insurance-Policy.pdf
All PDFs have been processed.


In [25]:
# Let's also check the length of all the texts as there might be some empty pages or pages with very few words that we can drop
insurance_pdfs_data['Text_Length'] = insurance_pdfs_data['Page_Text'].apply(lambda x: len(x.split(' ')))
display(insurance_pdfs_data.head())

,Page No.,Page_Text,Document Name,Text_Length
0,Page 1,DOROTHEA GLAUSE S655 RHODE ISLAND JOHN DOE 01/...,Principal-Sample-Life-Insurance-Policy.pdf,30
1,Page 2,This page left blank intentionally,Principal-Sample-Life-Insurance-Policy.pdf,5
2,Page 3,POLICY RIDER GROUP INSURANCE POLICY NO: S655 C...,Principal-Sample-Life-Insurance-Policy.pdf,230
3,Page 4,This page left blank intentionally,Principal-Sample-Life-Insurance-Policy.pdf,5
4,Page 5,PRINCIPAL LIFE INSURANCE COMPANY (called The P...,Principal-Sample-Life-Insurance-Policy.pdf,110


In [26]:
insurance_pdfs_data

,Page No.,Page_Text,Document Name,Text_Length
0,Page 1,DOROTHEA GLAUSE S655 RHODE ISLAND JOHN DOE 01/...,Principal-Sample-Life-Insurance-Policy.pdf,30
1,Page 2,This page left blank intentionally,Principal-Sample-Life-Insurance-Policy.pdf,5
2,Page 3,POLICY RIDER GROUP INSURANCE POLICY NO: S655 C...,Principal-Sample-Life-Insurance-Policy.pdf,230
3,Page 4,This page left blank intentionally,Principal-Sample-Life-Insurance-Policy.pdf,5
4,Page 5,PRINCIPAL LIFE INSURANCE COMPANY (called The P...,Principal-Sample-Life-Insurance-Policy.pdf,110
...,...,...,...,...
59,Page 60,I f a Dependent who was insured dies during th...,Principal-Sample-Life-Insurance-Policy.pdf,285
60,Page 61,Section D - Claim Procedures Article 1 - Notic...,Principal-Sample-Life-Insurance-Policy.pdf,418
61,Page 62,A claimant may request an appeal of a claim de...,Principal-Sample-Life-Insurance-Policy.pdf,322
62,Page 63,This page left blank intentionally,Principal-Sample-Life-Insurance-Policy.pdf,5


In [27]:
# Let's also check the length of all the texts as there might be some empty pages or pages with very few words that we can drop
insurance_pdfs_data = insurance_pdfs_data.loc[insurance_pdfs_data['Text_Length'] >= 10]
insurance_pdfs_data

,Page No.,Page_Text,Document Name,Text_Length
0,Page 1,DOROTHEA GLAUSE S655 RHODE ISLAND JOHN DOE 01/...,Principal-Sample-Life-Insurance-Policy.pdf,30
2,Page 3,POLICY RIDER GROUP INSURANCE POLICY NO: S655 C...,Principal-Sample-Life-Insurance-Policy.pdf,230
4,Page 5,PRINCIPAL LIFE INSURANCE COMPANY (called The P...,Principal-Sample-Life-Insurance-Policy.pdf,110
5,Page 6,TABLE OF CONTENTS PART I - DEFINITIONS PART II...,Principal-Sample-Life-Insurance-Policy.pdf,153
6,Page 7,Section A – Eligibility Member Life Insurance ...,Principal-Sample-Life-Insurance-Policy.pdf,176
7,Page 8,Section A - Member Life Insurance Schedule of ...,Principal-Sample-Life-Insurance-Policy.pdf,171
8,Page 9,P ART I - DEFINITIONS When used in this Group ...,Principal-Sample-Life-Insurance-Policy.pdf,387
9,Page 10,T he legally recognized union of two eligible ...,Principal-Sample-Life-Insurance-Policy.pdf,251
10,Page 11,(2) has been placed with the Member or spouse ...,Principal-Sample-Life-Insurance-Policy.pdf,299
11,Page 12,An institution that is licensed as a Hospital ...,Principal-Sample-Life-Insurance-Policy.pdf,352


In [28]:
insurance_pdfs_data['Metadata'] = insurance_pdfs_data.apply(lambda x: {'Policy_Name': x['Document Name'][:-4], 'Page_No.': x['Page No.']}, axis=1)

/tmp/ipython-input-2653819494.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  insurance_pdfs_data['Metadata'] = insurance_pdfs_data.apply(lambda x: {'Policy_Name': x['Document Name'][:-4], 'Page_No.': x['Page No.']}, axis=1)


In [29]:
insurance_pdfs_data

,Page No.,Page_Text,Document Name,Text_Length,Metadata
0,Page 1,DOROTHEA GLAUSE S655 RHODE ISLAND JOHN DOE 01/...,Principal-Sample-Life-Insurance-Policy.pdf,30,{'Policy_Name': 'Principal-Sample-Life-Insuran...
2,Page 3,POLICY RIDER GROUP INSURANCE POLICY NO: S655 C...,Principal-Sample-Life-Insurance-Policy.pdf,230,{'Policy_Name': 'Principal-Sample-Life-Insuran...
4,Page 5,PRINCIPAL LIFE INSURANCE COMPANY (called The P...,Principal-Sample-Life-Insurance-Policy.pdf,110,{'Policy_Name': 'Principal-Sample-Life-Insuran...
5,Page 6,TABLE OF CONTENTS PART I - DEFINITIONS PART II...,Principal-Sample-Life-Insurance-Policy.pdf,153,{'Policy_Name': 'Principal-Sample-Life-Insuran...
6,Page 7,Section A – Eligibility Member Life Insurance ...,Principal-Sample-Life-Insurance-Policy.pdf,176,{'Policy_Name': 'Principal-Sample-Life-Insuran...
7,Page 8,Section A - Member Life Insurance Schedule of ...,Principal-Sample-Life-Insurance-Policy.pdf,171,{'Policy_Name': 'Principal-Sample-Life-Insuran...
8,Page 9,P ART I - DEFINITIONS When used in this Group ...,Principal-Sample-Life-Insurance-Policy.pdf,387,{'Policy_Name': 'Principal-Sample-Life-Insuran...
9,Page 10,T he legally recognized union of two eligible ...,Principal-Sample-Life-Insurance-Policy.pdf,251,{'Policy_Name': 'Principal-Sample-Life-Insuran...
10,Page 11,(2) has been placed with the Member or spouse ...,Principal-Sample-Life-Insurance-Policy.pdf,299,{'Policy_Name': 'Principal-Sample-Life-Insuran...
11,Page 12,An institution that is licensed as a Hospital ...,Principal-Sample-Life-Insurance-Policy.pdf,352,{'Policy_Name': 'Principal-Sample-Life-Insuran...


In [30]:
from sentence_transformers import SentenceTransformer

# Load a Sentence-BERT model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [31]:
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

In [32]:
chroma_data_path = '/content/Principal-Sample-Life-Insurance-Policy.pdf/ChromaDB_Data'

In [33]:
import chromadb

In [34]:
client = chromadb.PersistentClient()

In [36]:
from chromadb.api.types import Documents, EmbeddingFunction, Embeddings
from typing import List

class SentenceTransformerEmbeddingFunction(EmbeddingFunction):
    def __init__(self, model_name: str = 'all-MiniLM-L6-v2'):
        self._model = SentenceTransformer(model_name)

    def __call__(self, texts: Documents) -> Embeddings:
        return self._model.encode(list(texts), convert_to_list=True)

# Instantiate the wrapper class
embedding_function = SentenceTransformerEmbeddingFunction()

# Now, create the collection using the wrapper instance
insurance_collection = client.get_or_create_collection(name='RAG_on_Insurance', embedding_function=embedding_function)

In [37]:
documents_list = insurance_pdfs_data["Page_Text"].tolist()
metadata_list = insurance_pdfs_data['Metadata'].tolist()

In [38]:

insurance_collection.add(
    documents= documents_list,
    ids = [str(i) for i in range(0, len(documents_list))],
    metadatas = metadata_list
)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [39]:

insurance_collection.get(
    ids = ['0','1','2'],
    include = ['embeddings', 'documents', 'metadatas']
)

{'ids': ['0', '1', '2'],
 'embeddings': array([[-0.0259219 ,  0.04777751,  0.05585773, ..., -0.04932661,
         -0.05851147,  0.02355199],
        [-0.10579571, -0.00053023,  0.01657757, ..., -0.03772428,
          0.03662254, -0.04041128],
        [-0.0635408 ,  0.05870159,  0.07262772, ..., -0.07400384,
         -0.02375576, -0.0081061 ]]),
 'documents': ['DOROTHEA GLAUSE S655 RHODE ISLAND JOHN DOE 01/01/2014 711 HIGH STREET GEORGE RI 02903 GROUP POLICY FOR: RHODE ISLAND JOHN DOE ALL MEMBERS Group Member Life Insurance Print Date: 07/16/2014',
  'POLICY RIDER GROUP INSURANCE POLICY NO: S655 COVERAGE: Life EMPLOYER: RHODE ISLAND JOHN DOE Effective on the later of the Date of Issue of this Group Policy or March 1, 2005, the following will apply to your Policy: From time to time The Principal may offer or provide certain employer groups who apply for coverage with The Principal a Financial Services Hotline and Grief Support Services or any other value added service for the employees o

In [40]:
cache_collection = client.get_or_create_collection(name='Insurance_Cache', embedding_function=embedding_function)

In [41]:
cache_collection.peek()

{'ids': [],
 'embeddings': array([], dtype=float64),
 'documents': [],
 'uris': None,
 'included': ['metadatas', 'documents', 'embeddings'],
 'data': None,
 'metadatas': []}

In [42]:
query = input()

what are policy changes


In [43]:

cache_results = cache_collection.query(
    query_texts=query,
    n_results=1
)

In [44]:
cache_results

{'ids': [[]],
 'embeddings': None,
 'documents': [[]],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[]],
 'distances': [[]]}

In [45]:
results = insurance_collection.query(
query_texts=query,
n_results=10
)

In [46]:
results.item()

AttributeError: 'dict' object has no attribute 'item'

In [50]:
# Implementing Cache in Semantic Search

# Set a threshold for cache search
threshold = 0.2

ids = []
documents = []
distances = []
metadatas = []
results_df = pd.DataFrame()


# If the distance is greater than the threshold, then return the results from the main collection.
# Also check if the cache_results is empty.
if not cache_results['distances'][0] or cache_results['distances'][0][0] > threshold:
      # Query the collection against the user query and return the top 10 results
      results = insurance_collection.query(
      query_texts=query,
      n_results=10
      )

      # Store the query in cache_collection as document w.r.t to ChromaDB so that it can be embedded and searched against later
      # Store retrieved text, ids, distances and metadatas in cache_collection as metadatas, so that they can be fetched easily if a query indeed matches to a query in cache

      # Prepare metadata to store in cache - converting lists to JSON strings
      cache_metadata = {
          "retrieved_ids": json.dumps(results["ids"][0]),
          "retrieved_documents": json.dumps(results["documents"][0]),
          "retrieved_distances": json.dumps(results["distances"][0]),
          "retrieved_metadatas": json.dumps(results["metadatas"][0])
      }

      cache_collection.add(
          documents=[query],
          ids=[query],
          metadatas=cache_metadata
      )

      print("Not found in cache. Found in main collection.")

      result_dict = {'Metadatas': results['metadatas'][0], 'Documents': results['documents'][0], 'Distances': results['distances'][0], "IDs":results["ids"][0]}
      results_df = pd.DataFrame.from_dict(result_dict)
      display(results_df)


# If the distance is, however, less than the threshold, you can return the results from cache
elif cache_results['distances'][0][0] <= threshold:
      cache_result_metadata = cache_results['metadatas'][0][0]

      # Load the JSON strings back into lists
      ids = json.loads(cache_result_metadata['retrieved_ids'])
      documents = json.loads(cache_result_metadata['retrieved_documents'])
      distances = json.loads(cache_result_metadata['retrieved_distances'])
      metadatas = json.loads(cache_result_metadata['retrieved_metadatas'])


      print("Found in cache!")

      # Create a DataFrame
      results_df = pd.DataFrame({
        'IDs': ids,
        'Documents': documents,
        'Distances': distances,
        'Metadatas': metadatas
      })
      display(results_df)

Not found in cache. Found in main collection.


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


,Metadatas,Documents,Distances,IDs
0,{'Policy_Name': 'Principal-Sample-Life-Insuran...,PART II - POLICY ADMINISTRATION Section A - Co...,0.781967,13
1,{'Policy_Name': 'Principal-Sample-Life-Insuran...,b . on any date the definition of Member or De...,1.045372,18
2,{'Policy_Name': 'Principal-Sample-Life-Insuran...,Section D - Policy Renewal Article 1 - Renewal...,1.061991,22
3,{'Policy_Name': 'Principal-Sample-Life-Insuran...,T he Principal has complete discretion to cons...,1.118748,16
4,"{'Page_No.': 'Page 6', 'Policy_Name': 'Princip...",TABLE OF CONTENTS PART I - DEFINITIONS PART II...,1.150761,3
5,{'Policy_Name': 'Principal-Sample-Life-Insuran...,Scheduled Benefit in force for the Member befo...,1.170646,28
6,"{'Page_No.': 'Page 24', 'Policy_Name': 'Princi...",T he Principal may terminate the Policyholder'...,1.226306,21
7,{'Policy_Name': 'Principal-Sample-Life-Insuran...,"(6) If, on the date a Member becomes eligible ...",1.227252,27
8,{'Policy_Name': 'Principal-Sample-Life-Insuran...,Section E - Reinstatement Article 1 - Reinstat...,1.251692,37
9,{'Policy_Name': 'Principal-Sample-Life-Insuran...,a . In no event will Dependent Life Insurance ...,1.276506,30


In [51]:
results_df

,Metadatas,Documents,Distances,IDs
0,{'Policy_Name': 'Principal-Sample-Life-Insuran...,PART II - POLICY ADMINISTRATION Section A - Co...,0.781967,13
1,{'Policy_Name': 'Principal-Sample-Life-Insuran...,b . on any date the definition of Member or De...,1.045372,18
2,{'Policy_Name': 'Principal-Sample-Life-Insuran...,Section D - Policy Renewal Article 1 - Renewal...,1.061991,22
3,{'Policy_Name': 'Principal-Sample-Life-Insuran...,T he Principal has complete discretion to cons...,1.118748,16
4,"{'Page_No.': 'Page 6', 'Policy_Name': 'Princip...",TABLE OF CONTENTS PART I - DEFINITIONS PART II...,1.150761,3
5,{'Policy_Name': 'Principal-Sample-Life-Insuran...,Scheduled Benefit in force for the Member befo...,1.170646,28
6,"{'Page_No.': 'Page 24', 'Policy_Name': 'Princi...",T he Principal may terminate the Policyholder'...,1.226306,21
7,{'Policy_Name': 'Principal-Sample-Life-Insuran...,"(6) If, on the date a Member becomes eligible ...",1.227252,27
8,{'Policy_Name': 'Principal-Sample-Life-Insuran...,Section E - Reinstatement Article 1 - Reinstat...,1.251692,37
9,{'Policy_Name': 'Principal-Sample-Life-Insuran...,a . In no event will Dependent Life Insurance ...,1.276506,30


In [52]:
from sentence_transformers import CrossEncoder, util

In [53]:

cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

In [54]:
scores = cross_encoder.predict([['Does the insurance cover diabetic patients?', 'The insurance policy covers some pre-existing conditions including diabetes, heart diseases, etc. The policy does not howev'],
                                ['Does the insurance cover diabetic patients?', 'The premium rates for various age groups are given as follows. Age group (<18 years): Premium rate']])

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [55]:
scores

array([  3.8467617, -11.252879 ], dtype=float32)

In [56]:
cross_inputs = [[query, response] for response in results_df['Documents']]
cross_rerank_scores = cross_encoder.predict(cross_inputs)

In [57]:
cross_rerank_scores

array([ 1.9920671 ,  0.31304023, -6.0750074 , -6.764147  ,  0.25127837,
       -1.1940905 , -6.3114824 , -2.7744436 , -7.382719  , -4.6499453 ],
      dtype=float32)

In [58]:
results_df['Reranked_scores'] = cross_rerank_scores

In [59]:

top_3_semantic = results_df.sort_values(by='Distances')
top_3_semantic[:3]

,Metadatas,Documents,Distances,IDs,Reranked_scores
0,{'Policy_Name': 'Principal-Sample-Life-Insuran...,PART II - POLICY ADMINISTRATION Section A - Co...,0.781967,13,1.992067
1,{'Policy_Name': 'Principal-Sample-Life-Insuran...,b . on any date the definition of Member or De...,1.045372,18,0.313040
2,{'Policy_Name': 'Principal-Sample-Life-Insuran...,Section D - Policy Renewal Article 1 - Renewal...,1.061991,22,-6.075007


In [60]:

top_3_rerank = results_df.sort_values(by='Reranked_scores', ascending=False)
top_3_rerank[:3]

,Metadatas,Documents,Distances,IDs,Reranked_scores
0,{'Policy_Name': 'Principal-Sample-Life-Insuran...,PART II - POLICY ADMINISTRATION Section A - Co...,0.781967,13,1.992067
1,{'Policy_Name': 'Principal-Sample-Life-Insuran...,b . on any date the definition of Member or De...,1.045372,18,0.313040
4,"{'Page_No.': 'Page 6', 'Policy_Name': 'Princip...",TABLE OF CONTENTS PART I - DEFINITIONS PART II...,1.150761,3,0.251278


In [61]:
top_3_RAG = top_3_rerank[["Documents", "Metadatas"]][:3]

In [62]:
# Define the function to generate the response. Provide a comprehensive prompt that passes the user query and the top 3 results to the model

def generate_response(query, results_df):
    """
    Generate a response using GPT-3.5's ChatCompletion based on the user query and retrieved information.
    """
    messages = [
                {"role": "system", "content":  "You are a helpful assistant in the insurance domain who can effectively answer user queries about insurance policies and documents."},
                {"role": "user", "content": f"""You are a helpful assistant in the insurance domain who can effectively answer user queries about insurance policies and documents.
                                                You have a question asked by the user in '{query}' and you have some search results from a corpus of insurance documents in the dataframe '{top_3_RAG}'. These search results are essentially one page of an insurance document that may be relevant to the user query.

                                                The column 'documents' inside this dataframe contains the actual text from the policy document and the column 'metadata' contains the policy name and source page. The text inside the document may also contain tables in the format of a list of lists where each of the nested lists indicates a row.

                                                Use the documents in '{top_3_RAG}' to answer the query '{query}'. Frame an informative answer and also, use the dataframe to return the relevant policy names and page numbers as citations.

                                                Follow the guidelines below when performing the task.
                                                1. Try to provide relevant/accurate numbers if available.
                                                2. You don’t have to necessarily use all the information in the dataframe. Only choose information that is relevant.
                                                3. If the document text has tables with relevant information, please reformat the table and return the final information in a tabular in format.
                                                3. Use the Metadatas columns in the dataframe to retrieve and cite the policy name(s) and page numbers(s) as citation.
                                                4. If you can't provide the complete answer, please also provide any information that will help the user to search specific sections in the relevant cited documents.
                                                5. You are a customer facing assistant, so do not provide any information on internal workings, just answer the query directly.

                                                The generated response should answer the query directly addressing the user and avoiding additional information. If you think that the query is not relevant to the document, reply that the query is irrelevant. Provide the final response as a well-formatted and easily readable text along with the citation. Provide your complete response first with all information, and then provide the citations.
                                                """},
              ]

    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages
    )

    return response.choices[0].message.content.split('\n')

In [64]:
results_df

,Metadatas,Documents,Distances,IDs,Reranked_scores
0,{'Policy_Name': 'Principal-Sample-Life-Insuran...,PART II - POLICY ADMINISTRATION Section A - Co...,0.781967,13,1.992067
1,{'Policy_Name': 'Principal-Sample-Life-Insuran...,b . on any date the definition of Member or De...,1.045372,18,0.313040
2,{'Policy_Name': 'Principal-Sample-Life-Insuran...,Section D - Policy Renewal Article 1 - Renewal...,1.061991,22,-6.075007
3,{'Policy_Name': 'Principal-Sample-Life-Insuran...,T he Principal has complete discretion to cons...,1.118748,16,-6.764147
4,"{'Page_No.': 'Page 6', 'Policy_Name': 'Princip...",TABLE OF CONTENTS PART I - DEFINITIONS PART II...,1.150761,3,0.251278
5,{'Policy_Name': 'Principal-Sample-Life-Insuran...,Scheduled Benefit in force for the Member befo...,1.170646,28,-1.194090
6,"{'Page_No.': 'Page 24', 'Policy_Name': 'Princi...",T he Principal may terminate the Policyholder'...,1.226306,21,-6.311482
7,{'Policy_Name': 'Principal-Sample-Life-Insuran...,"(6) If, on the date a Member becomes eligible ...",1.227252,27,-2.774444
8,{'Policy_Name': 'Principal-Sample-Life-Insuran...,Section E - Reinstatement Article 1 - Reinstat...,1.251692,37,-7.382719
9,{'Policy_Name': 'Principal-Sample-Life-Insuran...,a . In no event will Dependent Life Insurance ...,1.276506,30,-4.649945


In [69]:
from transformers import pipeline

# Load the model and tokenizer
# Using a smaller, publicly available model for demonstration
model_name = "distilbert/distilgpt2"
qa_pipeline = pipeline(
    "text-generation",
    model=model_name,
    device=0  # Use 0 for GPU or -1 for CPU
)

# Get the maximum sequence length for the loaded model
max_seq_length = qa_pipeline.model.config.max_position_embeddings

def generate_response(query, results_df):
    # Ensure results_df is not empty and contains the expected columns
    if results_df.empty or 'Documents' not in results_df.columns or 'Metadatas' not in results_df.columns:
        return ["Could not retrieve relevant information to answer the query."]

    context = "\n\n".join([
        f"Document: {row['Documents']}\nSource: Policy: {row['Metadatas'].get('Policy_Name', 'N/A')}, Page: {row['Metadatas'].get('Page_No.', 'N/A')}" for _, row in results_df.iterrows()
    ])

    # Truncate the context if it exceeds the maximum sequence length,
    # reserving some space for the query and prompt template
    # A rough estimation: allocate 80% for context and 20% for query/template
    max_context_length = int(max_seq_length * 0.8) - len(query) - 100 # Subtract some buffer

    if len(context) > max_context_length:
        context = context[:max_context_length] + "..." # Truncate and add ellipsis

    prompt = f"""
You are a helpful assistant in the insurance domain who can effectively answer user queries about insurance policies and documents.

User Query: {query}

Based on the following documents, provide a clear, helpful, and concise answer. Include relevant numbers or table information if available. Also, cite the policy name and page number from the metadata at the end of your answer.

Documents:
{context}
"""
    # Generating text with a causal language model pipeline
    # max_new_tokens controls the maximum length of the generated text
    # do_sample=True enables sampling, which can lead to more varied outputs
    output = qa_pipeline(prompt, max_new_tokens=200, do_sample=True)[0]['generated_text']

    # Post-process the output to extract the answer and citations if necessary
    # For a simple text generation model, you might need to parse the output
    # to separate the answer from any generated prompt or extra text.
    # For this example, we'll return the generated text directly and add citations separately.

    return [output]

Device set to use cpu


In [65]:
from transformers import pipeline

In [70]:
response = generate_response(query, top_3_RAG)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [71]:
print("\n".join(response))


You are a helpful assistant in the insurance domain who can effectively answer user queries about insurance policies and documents.

User Query: what are policy changes

Based on the following documents, provide a clear, helpful, and concise answer. Include relevant numbers or table information if available. Also, cite the policy name and page number from the metadata at the end of your answer.

Documents:
Document: PART II - POLICY ADMINISTRATION Section A - Contract Article 1 - Entire Contract This Group Policy, the current Certificate, the attached Policyholder application, and any Member applications make up the entire contract. The Principal is obligated only as provided in this Group Policy and is not bound by any trust or plan to which it is not a signatory party. Article 2 - Policy Changes Insurance under this Group Policy runs annually to the Policy Anniversary, unless sooner terminated. No agent, employee, or person other than an officer of The Principal has authority to cha